<center><img src="https://github.com/hse-ds/iad-applied-ds/blob/master/2021/hw/hw1/img/logo_hse.png?raw=1" width="1000"></center>

<h1><center>Прикладные задачи анализа данных</center></h1>
<h2><center>Домашнее задание 4: рекомендательные системы</center></h2>

# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [36]:
import pandas as pd
import numpy as np
import math
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from bayes_opt import BayesianOptimization

Загрузим данные и проведем предобраотку данных как на семинаре.

In [2]:
# !kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
# !unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

In [3]:
import os
os.listdir('/kaggle/input/articles-sharing-reading-from-cit-deskdrop/')

['shared_articles.csv', 'users_interactions.csv']

In [4]:
articles_df = pd.read_csv("/kaggle/input/articles-sharing-reading-from-cit-deskdrop/shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [5]:
interactions_df = pd.read_csv("/kaggle/input/articles-sharing-reading-from-cit-deskdrop/users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [6]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [7]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   "VIEW": 1.0,
   "LIKE": 2.0, 
   "BOOKMARK": 2.5, 
   "FOLLOW": 3.0,
   "COMMENT CREATED": 4.0,  
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [8]:
users_interactions_count_df = (
    interactions_df
    .groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId").size())
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[["personId"]]
print("# users with at least 5 interactions:",len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [9]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [10]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [11]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"]).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"])["timestamp"].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [12]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [13]:
interactions = (
    interactions_train_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = (
    interactions_test_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    "" for x in range(len(interactions.loc[pd.isnull(interactions.true_test), "true_test"]))]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [14]:
!pip install lightfm

In [15]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Задание 1 (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [16]:
# функция создания разреженной матрицы
def create_sp_matrix(dataframe, users, contents):    
    m = pd.DataFrame(0, columns=contents, index=users)
    contentIds, personIDs, eventStrengths = dataframe['contentId'].values, dataframe['personId'].values, dataframe['eventStrength'].values
    n = len(dataframe)
    for i in range(n):
        if eventStrengths[i] is not np.nan:
            m.loc[personIDs[i], contentIds[i]] = eventStrengths[i] 
        else:
            m.loc[personIDs[i], contentIds[i]] = 0
    result = csr_matrix(m.values)
    return result

content_dataframe_united = pd.concat([interactions_train_df['contentId'], interactions_test_df['contentId']])
persons_dataframe_united = pd.concat([interactions_train_df['personId'], interactions_test_df['personId']])
sumContentIds, sumPersonIds = set(content_dataframe_united), set(persons_dataframe_united)

In [17]:
# Ваш код здесь
data_train = create_sp_matrix(interactions_train_df,sumPersonIds, sumContentIds)
data_test = create_sp_matrix(interactions_test_df,sumPersonIds, sumContentIds)

In [18]:
np.isnan(data_train.data).any()
np.isnan(data_test.data).any()
data_train.data = np.nan_to_num(data_train.data)
data_test.data = np.nan_to_num(data_test.data)
data_train.eliminate_zeros()
data_test.eliminate_zeros()

## Задание 2 (1 балл)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [19]:
# Ваш код здесь
my_model = LightFM(loss="warp")
my_model.fit(data_train, epochs=50)
train_prec_mean = precision_at_k(my_model, data_train, k=10).mean()
test_prec_mean = precision_at_k(my_model, data_test, k=10, train_interactions=data_train).mean()
print("Precision on train data:", train_prec_mean)
print("Precision on test data:", test_prec_mean)

Precision on train data: 0.2596223
Precision on test data: 0.0070264763


## Задание 3 (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [20]:
# Ваш код здесь
my_vectorizer_model = TfidfVectorizer()
feat = my_vectorizer_model.fit_transform(articles_df['text'])
print(feat.shape)
my_model_2 = LightFM(loss="warp")
my_model_2.fit(data_train, item_features=feat, epochs=15)
train_prec_mean_2 = precision_at_k(my_model_2, data_train, item_features=feat, k=10).mean()
test_prec_mean_2 = precision_at_k(my_model_2, data_test, item_features=feat, k=10, train_interactions=data_train).mean()
print("Precision on train data:", train_prec_mean_2)
print("Precision on test data:", test_prec_mean_2)

(3047, 72615)
Precision on train data: 0.23309353
Precision on test data: 0.006517312


## Задание 4 (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [21]:
# Ваш код здесь
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords 
nltk.download('stopwords')
stop_word_list = stopwords.words('english')  #  + stopwords.words('portuguese')
my_stemmer = PorterStemmer()

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [22]:
def preprocess_text(my_str):    
    words = re.sub(r"[^A-Za-z0-9\-]", " ", my_str).lower().split()
    words = [my_stemmer.stem(word) for word in words if word not in stop_word_list]
    return words

In [23]:
my_vectorizer_model_2 = TfidfVectorizer(analyzer='word', tokenizer=preprocess_text)
feat_2 = my_vectorizer_model_2.fit_transform(articles_df['text'])

In [24]:
my_model_3 = LightFM(loss="warp")
my_model_3.fit(data_train, item_features=feat_2, epochs=15)
train_prec_mean_3 = precision_at_k(my_model_3, data_train, item_features=feat_2, k=10).mean()
test_prec_mean_3 = precision_at_k(my_model_3, data_test, item_features=feat_2, k=10, train_interactions=data_train).mean()
print("Precision on train data:", train_prec_mean_3)
print("Precision on test data:", test_prec_mean_3)

Precision on train data: 0.22419067
Precision on test data: 0.005702648


Улучшилось ли качество предсказания?

Качество осталось примерно таким же((

## Задание 5 (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [25]:
# Ваш код здесь
def searching_best_params(components_num, epoch_num, max_sampled):
    my_model = LightFM(loss="warp", no_components=int(components_num), max_sampled=max_sampled)
    my_model.fit(data_train, item_features=feat_2, epochs=int(epoch_num))
    return(precision_at_k(my_model, data_test, item_features=feat_2, k=10, train_interactions=data_train).mean()) # считаем качество на тестовой выборке

In [26]:
parameter_list = {
    'components_num': (10, 30), 
    "max_sampled": (5, 15),
    "epoch_num": (15, 30)}
my_optimizer = BayesianOptimization(f=searching_best_params, pbounds=parameter_list, random_state=12345)
my_optimizer.maximize(init_points=4, n_iter=5)
# n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
# init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

|   iter    |  target   | compon... | epoch_num | max_sa... |
-------------------------------------------------------------
|  1        |  0.007943 |  28.59    |  19.75    |  6.839    |
|  2        |  0.00611  |  14.09    |  23.52    |  10.96    |
|  3        |  0.006517 |  29.29    |  24.8     |  12.49    |
|  4        |  0.006008 |  23.07    |  26.22    |  14.61    |
|  5        |  0.006823 |  29.65    |  17.01    |  5.461    |
|  6        |  0.006314 |  27.13    |  20.11    |  6.828    |
|  7        |  0.006721 |  10.52    |  23.61    |  13.38    |
|  8        |  0.006212 |  13.8     |  22.85    |  7.072    |
|  9        |  0.006619 |  13.04    |  27.85    |  5.113    |


In [27]:
print(my_optimizer.max)

{'target': 0.007942973636090755, 'params': {'components_num': 28.592321856342956, 'epoch_num': 19.74563331872679, 'max_sampled': 6.839188116770945}}


Обучим с лучшими параметрами

In [28]:
my_model_best = LightFM(loss="warp", no_components=int(my_optimizer.max['params']['components_num']), 
                     max_sampled=int(my_optimizer.max['params']['max_sampled']))
my_model_best.fit(data_train, item_features=feat_2, epochs=int(my_optimizer.max['params']['epoch_num']))
train_prec_mean_best = precision_at_k(my_model_best, data_train, item_features=feat_2, k=10).mean()
test_prec_mean_best = precision_at_k(my_model_best, data_test, item_features=feat_2, k=10, train_interactions=data_train).mean()
print("Precision on train data:", train_prec_mean_best)
print("Precision on test data:", test_prec_mean_best)

Precision on train data: 0.340018
Precision on test data: 0.0059063137


## Бонусное задание (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

In [ ]:
# Ваш код здесь